In [ ]:
import pandas
from selenium import webdriver
import time
from bs4 import BeautifulSoup as bs
import warnings
warnings.filterwarnings('ignore')

path= 'C:\Program Files (x86)\chromedriver.exe'

driver= webdriver.Chrome(path)

data= {"product_name":list(),"product_url":list(),"status":list(),"stars":list()}

def scrapper(url):
    base_url='https://yoshops.com'
    driver.get(url)
    time.sleep(5)
    html= driver.page_source
    soup= bs(html,'html.parser')
    products= soup.find_all("div",{"class":"product"})
    for i in range(len(products)):
        obj= products[i].find("div",{"class":"single-product-review"}).span
        if obj is None:
            data["product_name"].append(products[i].a.span.text.strip())
            data["product_url"].append(base_url + str(products[i].a['href']))
            data['status'].append("Not Reviewed")
            data['stars'].append("0 Star")
        else:
            data["product_name"].append(products[i].a.span.text.strip())
            data["product_url"].append(base_url + str(products[i].a['href']))
            data['status'].append("Reviewed")
            data['stars'].append(obj.text.strip())
def all_pages():
    url = "https://yoshops.com/products?page="
    allpages = [url + str(i) for i in range(0, 12)]
    return allpages

def get_categories():
    driver.get("https://yoshops.com/t/laptops")
    html = driver.page_source
    soup = bs(html, 'html.parser')
    cet = soup.find_all('div', {"class": "categories"})
    cet_anchores = cet[0].find_all('a')
    cate_dict = {}
    base_url = 'https://yoshops.com'
    for a in cet_anchores:
        cate_dict[a.text.strip()] = base_url + str(a['href'])
    return cate_dict


def for_next_page():
    try:
        pages = driver.find_element_by_class_name('pagination')
        an = pages.find_elements_by_tag_name('a')
        li = []
        for a in an:
            li.append(a.get_attribute('href'))
        for page_url in li[2:-1]:
            scrapper(page_url)

    except:
        pass
def scrapper_for_not_reviewed(url):
    base_url='https://yoshops.com'
    driver.get(url)
    time.sleep(5)
    html= driver.page_source
    soup= bs(html,'html.parser')
    products= soup.find_all("div",{"class":"product"})
    for i in range(len(products)):
        obj= products[i].find("div",{"class":"single-product-review"}).span
        if obj is None:
            data["product_name"].append(products[i].a.span.text.strip())
            data["product_url"].append(base_url + str(products[i].a['href']))
            data['status'].append("Not Reviewed")
            data['stars'].append("0 Star")
        else:
            pass

def scrapper_for_reviewed(url):
    base_url='https://yoshops.com'
    driver.get(url)
    time.sleep(5)
    html= driver.page_source
    soup= bs(html,'html.parser')
    products= soup.find_all("div",{"class":"product"})
    for i in range(len(products)):
        obj= products[i].find("div",{"class":"single-product-review"}).span
        if obj is None:
            pass
        else:
            data["product_name"].append(products[i].a.span.text.strip())
            data["product_url"].append(base_url + str(products[i].a['href']))
            data['status'].append("Reviewed")
            data['stars'].append(obj.text.strip())

def save_data(data,filename):
    df= pandas.DataFrame(data)
    df.to_csv(filename,index=False)

if __name__ == "__main__":

    task = input("Enter 1 for all products : "
                 "\nEnter 2 for category wise products :"
                 "\nEnter 3 for only reviewed products and not reviewed products :\n")
    if task == '1':
        pages= all_pages()
        for page in pages:
            scrapper(page)
        save_data(data,filename="All_Products_Review.csv")
        print("Successfull...")

    elif task == '2':
        cat= get_categories()
        for key in cat.keys():
            print(key)
        key= input("Enter category: ")
        link= cat[key]
        scrapper(link)
        for_next_page()
        save_data(data,filename=key + '.csv')

    elif task == '3':
        var= input("Enter 1 for only Reviewed Products: \n"
                   "Enter 2 for only Not Reviewed Products: \n")
        if var=='1':
            scrapper_for_reviewed(page)
            save_data(data,filename="Reviewed_Products.csv")
            print("Successfull...")
            
        elif var== '2':
            scrapper_for_not_reviewed(page)
            save_data(data, filename="Not_Reviewed_Products.csv")
            print("Successfull...")
        else:
            print("Not Successfull...")
            print("Enter correct input")
            

    else:
        print("Enter correct input")
        task = input("Enter 1 for all products : "
                 "\nEnter 2 for category wise products :"
                 "\nEnter 3 for only reviewed products and not reviewed products :\n")
        if task == '1':
            pages= all_pages()
            for page in pages:
                scrapper(page)
            save_data(data,filename="All_Products_Review.csv")
            print("Successfull...")

        elif task == '2':
            cat= get_categories()
            for key in cat.keys():
                print(key)
            key= input("Enter category: ")
            link= cat[key]
            scrapper(link)
            for_next_page()
            save_data(data,filename=key + '.csv')

        elif task == '3':
            var= input("Enter 1 for only Reviewed Products: \n"
                       "Enter 2 for only Not Reviewed Products: \n")
            if var=='1':
                scrapper_for_reviewed(page)
                save_data(data,filename="Reviewed_Products.csv")
                print("Successfull...")

            elif var== '2':
                scrapper_for_not_reviewed(page)
                save_data(data, filename="Not_Reviewed_Products.csv")
                print("Successfull...")
    driver.close()